In [5]:
from BenqAmazonScraper import BenqAmazonScraper
import pandas as pd
import pandas as pd
import requests
from parsel import Selector
from bs4 import BeautifulSoup

In [7]:
benqScraper = BenqAmazonScraper()
filename = 'test2.xlsx'
data = pd.read_excel(filename, usecols=[0])
print(data.head())

         ASIN
0  B0CNSB97QN
1  B01H5KKQTM


In [6]:
file_path = "test.xlsx"
excel_df = pd.DataFrame()
excel_df = pd.read_excel(file_path)
print(excel_df)
for index, row in excel_df.iterrows():
    baseLink = "https://www.amazon/dp/" + str(excel_df.iat[index, 0])
    excel_df.iat[index, 1] = baseLink
print(excel_df)

          ASIN  Price  Sold By   Fulfilled By
0   B0CNSB97QN    NaN       NaN           NaN
1   B01H5KKQTM    NaN       NaN           NaN
2   B08G57L985    NaN       NaN           NaN
3   B0B434LTLF    NaN       NaN           NaN
4   B08G57LF6X    NaN       NaN           NaN
..         ...    ...       ...           ...
83  B076CTL3BD    NaN       NaN           NaN
84  B0911822LZ    NaN       NaN           NaN
85  B0BZT3SLWV    NaN       NaN           NaN
86  B0BZSYR4YD    NaN       NaN           NaN
87  B08412P2GB    NaN       NaN           NaN

[88 rows x 4 columns]
          ASIN                             Price  Sold By   Fulfilled By
0   B0CNSB97QN  https://www.amazon/dp/B0CNSB97QN       NaN           NaN
1   B01H5KKQTM  https://www.amazon/dp/B01H5KKQTM       NaN           NaN
2   B08G57L985  https://www.amazon/dp/B08G57L985       NaN           NaN
3   B0B434LTLF  https://www.amazon/dp/B0B434LTLF       NaN           NaN
4   B08G57LF6X  https://www.amazon/dp/B08G57LF6X       NaN  

C:\Users\anthony.tran\AppData\Local\Temp\ipykernel_23972\1869427432.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'https://www.amazon/dp/B0CNSB97QN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  excel_df.iat[index, 1] = baseLink


In [22]:
baseLink1 = "https://www.amazon.com/dp/" + "B0CNSB97QN" #Ships from Amazon, Sold by BenQ INFTY Lab
baseLink2 = "https://www.amazon.com/dp/" + "B01H5KKQTM" #Currently unavailable
baseLink3 = "https://www.amazon.com/dp/" + "B08G57LF6X" #Ships from Adorama, Sold by Adorama
baseLink4 = "https://www.amazon.com/dp/" + "B0B434LTLF" #Ships from Benq, sold by Benq
baseLink5 = "https://www.amazon.com/dp/" + "B08G57L985" #Ships from INFTY, sold by INFTY
custom_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'accept-language': 'en-GB,en;q=0.9'
}
try:
    response = requests.get(baseLink5, headers=custom_header)
    if response.status_code == 200:
        sel = Selector(text=response.text)
        price = "$" + sel.css('.a-price-whole ::text').get("") + "." + sel.css('.a-price-fraction ::text').get("")
        # Try using beautiful soup
        soup = BeautifulSoup(response.text, 'html.parser')
        buybox = soup.find(id='desktop_buybox')
        buyboxSoup = BeautifulSoup(str(buybox), 'html.parser')
        shipsFrom = buyboxSoup.find(id='shipFromSoldByAbbreviated_feature_div')
        shipsFrom2 = buyboxSoup.find(id="sellerProfileTriggerId")

        availability_indicator = soup.find('span', string='Currently unavailable.')

        if availability_indicator:
            print("product is unavailable")
            
except Exception as e:
    print(e)

In [24]:
shipsFromInfo = BeautifulSoup(str(shipsFrom), 'html.parser')
span_elements = shipsFromInfo.find_all('span')

# shipsFrom shows Ships by
# shipsFrom2 shows Sold by
with open('text1.txt', 'w') as file:
    file.write(str(shipsFrom))

with open('text2.txt', 'w') as file:
    if (shipsFrom2):
        file.write("Sold by: " + str(shipsFrom2.text) + '\n') #Sold by
    else:
        file.write("Sold by: Amazon\n")
    if len(span_elements) == 0:
        file.write("Shipped From: Amazon\n") #Ships from
    else:
        index = 0
        while index < len(span_elements):
            if span_elements[index].text.strip() == "Ships from:":
                # Return this
                shipsFrom = str(span_elements[index + 1].text)
                file.write("Shipped From: " + shipsFrom) #Ships from
                index = len(span_elements)
            else:
                index += 1

print(shipsFrom2.text)#none on unavailable

BenQ INFTY Lab


In [38]:
# If not ship by Amazon
for span in span_elements:
    print(span.text)
index = 0
# shipsFrom = ""
# soldBy = ""
while index < len(span_elements):
    if span_elements[index].text.strip() == "Ships from:":
        # Return this
        shipsFrom = str(span_elements[index + 1].text)
        print(shipsFrom)
        index += 2
    elif span_elements[index].text.strip() == "Sold by:":
        soldBy = span_elements[index + 1].text
        print(soldBy)
        index += 2
    else:
        index += 1